In [158]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of xhydro.frequency_analysis.regional failed: Traceback (most recent call last):
  File "c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\PM217776\miniconda3\envs\xhydro-dev\Lib\importlib\__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 991, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1129, in get_code
  File "<frozen importlib._bootstrap_external>", line 1059, in source_to_code
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "C:\Python\xhydro-github\AFR_branch\xhydro\sr

In [159]:
import numpy as np
import pandas as pd
import xarray as xr
import xdatasets as xd
from sklearn.cluster import HDBSCAN, OPTICS, AgglomerativeClustering
from sklearn.decomposition import PCA

import xhydro as xh
import xhydro.frequency_analysis as xhfa
import xhydro.gis as xhgis

In [160]:
# TODO add multiple regions (bootstrap and param)
# TODO Add bootsrap
# TODO Document and explain
# TODO Unit test

In [161]:
from lmoments3.distr import KappaGen

from xhydro.frequency_analysis.regional import *

In [143]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["02*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

ds

URLError: <urlopen error Could not reach the catalog, perhaps due to the presence of a proxy.Try adding proxy information to the environment variables as follows beforerunning xdatasets :import osproxy = 'http://<proxy>:<port>'os.environ['http_proxy'] = proxyos.environ['https_proxy'] = proxy>

In [67]:
# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "annual": {},
}

In [68]:
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

ds_4fa

<xarray.Dataset> Size: 15kB
Dimensions:                (id: 27, time: 56)
Coordinates: (12/15)
    drainage_area          (id) float32 108B 1.09e+03 647.0 ... 2.163e+03 48.1
    end_date               (id) datetime64[ns] 216B 2006-10-13 ... 2024-05-28
  * id                     (id) object 216B '020302' '020404' ... '024015'
    latitude               (id) float32 108B 48.77 48.81 48.98 ... 46.2 46.18
    longitude              (id) float32 108B -64.52 -64.92 ... -72.1 -71.75
    name                   (id) object 216B 'Saint' 'York' ... 'Bourbon'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 216B 1989-08-12 ... 2006-07-24
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
Data variables:
    streamflow_max_spring  (id, time) float32 6kB nan nan nan ... 11.29 nan nan
    streamflow_max_annual  (id, time) float32 6kB nan nan nan ... 15.12 nan nan
Attributes:
    cat:variable:          ('streamflow_max_spring',)
    cat:xrfreq:            YS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [69]:
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)
ds_4fa

<xarray.Dataset> Size: 39kB
Dimensions:                (id: 27, time: 56)
Coordinates: (12/15)
    drainage_area          (id) float32 108B 1.09e+03 647.0 ... 2.163e+03 48.1
    end_date               (id) datetime64[ns] 216B 2006-10-13 ... 2024-05-28
  * id                     (id) object 216B '020302' '020404' ... '024015'
    latitude               (id) float32 108B 48.77 48.81 48.98 ... 46.2 46.18
    longitude              (id) float32 108B -64.52 -64.92 ... -72.1 -71.75
    name                   (id) object 216B 'Saint' 'York' ... 'Bourbon'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 216B 1989-08-12 ... 2006-07-24
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
Data variables:
    streamflow_max_spring  (id, time) float32 6kB nan nan nan ... 11.29 nan nan
    streamflow_max_annual  (id, time) float32 6kB nan nan nan ... 15.12 nan nan
    volume_sum_spring      (id, time) float64 12kB nan nan nan ... 1.907 nan nan
    volume_sum_annual      (id, time) float64 12kB nan nan nan ... 37.17 nan nan
Attributes:
    cat:variable:          ('streamflow_max_spring',)
    cat:xrfreq:            YS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [70]:
# Extraction des SHP pour les staton hydrométriques provinciales
gdf = xd.Query(
    **{
        "datasets": {
            "deh_polygons": {
                "id": ["02*"],
                "regulated": ["Natural"],
                "variables": ["streamflow"],
            }
        },
        "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
    }
).data.reset_index()
gdf

,Station,Superficie,geometry
0,020302,1071.505249,"POLYGON ((-65.54653 48.91282, -65.54639 48.912..."
1,020404,664.096924,"POLYGON ((-65.1477 49.05904, -65.14748 49.0589..."
2,020502,57.292057,"POLYGON ((-64.45703 48.9948, -64.45692 48.9946..."
3,020602,626.996155,"POLYGON ((-64.97292 49.17614, -64.97283 49.176..."
4,020802,1184.306641,"POLYGON ((-65.26495 49.21157, -65.26473 49.211..."
5,021407,763.051636,"POLYGON ((-66.01417 49.1084, -66.0141 49.1083,..."
6,021502,718.665344,"POLYGON ((-66.58245 49.05639, -66.58243 49.056..."
7,021915,484.327454,"POLYGON ((-68.17734 48.54264, -68.17717 48.542..."
8,021916,93.784286,"POLYGON ((-68.33648 48.42673, -68.33656 48.426..."
9,022507,515.224609,"POLYGON ((-69.6483 47.61478, -69.64672 47.6137..."


In [71]:
dswp = xhgis.watershed_properties(
    gdf[["Station", "geometry"]], unique_id="Station", output_format="xarray"
)
cent = dswp["centroid"].to_numpy()
lon = [ele[0] for ele in cent]
lat = [ele[1] for ele in cent]
dswp = dswp.assign(lon=("Station", lon))
dswp = dswp.assign(lat=("Station", lat))
dswp = dswp.drop("centroid")
dswp

<xarray.Dataset> Size: 1kB
Dimensions:    (Station: 27)
Coordinates:
  * Station    (Station) object 216B '020302' '020404' ... '024014' '024015'
Data variables:
    area       (Station) float64 216B 1.072e+09 6.641e+08 ... 2.17e+09 4.969e+07
    perimeter  (Station) float64 216B 3.637e+05 2.496e+05 ... 4.658e+04
    gravelius  (Station) float64 216B 3.134 2.732 1.747 ... 2.132 2.636 1.864
    lon        (Station) float64 216B -65.09 -65.24 -64.46 ... -71.56 -71.71
    lat        (Station) float64 216B 48.74 48.9 48.96 ... 46.08 46.19 46.16

In [72]:
data, pca = xhfa.regional.fit_pca(dswp, n_components=3)
data

<xarray.DataArray (Station: 27, components: 3)> Size: 648B
array([[ 2.67035877,  1.75919216,  0.76206293],
       [ 1.06732866,  2.05965305,  0.38237151],
       [-2.03313717,  2.72214445, -1.10333638],
       [ 0.23552091,  2.30462729, -0.480144  ],
       [ 2.28301367,  1.79640619,  0.05511961],
       [ 0.77279718,  1.8233214 , -0.16403192],
       [-0.41414625,  1.71936302, -1.26173939],
       [-0.16417996,  1.04900623,  0.018554  ],
       [-0.54279624,  1.18150351,  1.35063966],
       [-0.10839644,  0.06727432,  0.2350064 ],
       [-0.77847203,  0.17459775,  0.68645585],
       [ 0.45988053, -0.27645746,  0.02192669],
       [-2.01115906, -0.45214199,  0.07851317],
       [-1.17397731, -0.53728529,  0.48885036],
       [ 1.30791083, -1.06398715, -0.04599058],
       [ 0.78848687, -1.09594506,  0.86546179],
       [-0.22663565, -0.91830316, -0.29336544],
       [-0.09918532, -1.16208889, -0.49326336],
       [-0.97448487, -0.75541   ,  0.99270092],
       [-0.50970088, -1.10151845,  0.76274515],
       [-0.39477176, -1.11678297,  0.79860603],
       [ 0.33317106, -1.40198459, -0.24547967],
       [ 3.71540982, -2.04160684, -0.76785795],
       [-2.93993266, -0.84623387, -0.96361099],
       [-1.5661671 , -1.02568556, -0.09764733],
       [ 2.82535278, -1.91192701, -1.15679412],
       [-2.52208839, -0.94973106, -0.42575295]])
Coordinates:
  * Station     (Station) object 216B '020302' '020404' ... '024014' '024015'
  * components  (components) int32 12B 0 1 2

In [73]:
data.to_dataframe(name="value").reset_index().pivot(
    index="Station", columns="components"
).corr()

value                            
components                   0             1             2
      components                                          
value 0           1.000000e+00 -2.965021e-17 -1.579885e-16
      1          -2.965021e-17  1.000000e+00 -7.699615e-17
      2          -1.579885e-16 -7.699615e-17  1.000000e+00

In [74]:
pca.n_components_

3

In [75]:
xhfa.regional.get_group_from_fit(HDBSCAN, {"min_cluster_size": 2}, data)

[array(['024007', '024014'], dtype=object),
 array(['020302', '020802'], dtype=object),
 array(['020404', '020602', '021407', '021502'], dtype=object),
 array(['023303', '023401'], dtype=object),
 array(['023422', '023428', '024003'], dtype=object),
 array(['024010', '024015'], dtype=object),
 array(['023002', '024013'], dtype=object),
 array(['022507', '022601', '022704'], dtype=object),
 array(['023004', '023432'], dtype=object),
 array(['023701', '023702'], dtype=object)]

In [76]:
xhfa.regional.get_group_from_fit(OPTICS, {"min_samples": 2}, data)

[array(['020302', '020802'], dtype=object),
 array(['020404', '020602', '021407'], dtype=object),
 array(['022507', '022704'], dtype=object),
 array(['023004', '023432'], dtype=object),
 array(['023701', '023702'], dtype=object),
 array(['023002', '024013'], dtype=object),
 array(['024010', '024015'], dtype=object),
 array(['023422', '023428', '024003'], dtype=object),
 array(['024007', '024014'], dtype=object)]

In [77]:
groupes = xhfa.regional.get_group_from_fit(
    AgglomerativeClustering, {"n_clusters": 3}, data
)
groupes

[array(['022507', '022601', '022704', '023002', '023004', '023303',
        '023401', '023422', '023428', '023432', '023701', '023702',
        '024003', '024010', '024013', '024015'], dtype=object),
 array(['020302', '020404', '020502', '020602', '020802', '021407',
        '021502', '021915', '021916'], dtype=object),
 array(['024007', '024014'], dtype=object)]

In [79]:
ds_moment = xr.apply_ufunc(
    moment_l_vector, ds_4fa, input_core_dims=[["time"]], output_core_dims=[["lmom"]]
).assign_coords(lmom=["l1", "l2", "l3", "tau", "tau3", "tau4"])
ds_moment

<xarray.Dataset> Size: 7kB
Dimensions:                (id: 27, lmom: 6)
Coordinates: (12/15)
    drainage_area          (id) float32 108B 1.09e+03 647.0 ... 2.163e+03 48.1
    end_date               (id) datetime64[ns] 216B 2006-10-13 ... 2024-05-28
  * id                     (id) object 216B '020302' '020404' ... '024015'
    latitude               (id) float32 108B 48.77 48.81 48.98 ... 46.2 46.18
    longitude              (id) float32 108B -64.52 -64.92 ... -72.1 -71.75
    name                   (id) object 216B 'Saint' 'York' ... 'Bourbon'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 216B 1989-08-12 ... 2006-07-24
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * lmom                   (lmom) <U4 96B 'l1' 'l2' 'l3' 'tau' 'tau3' 'tau4'
Data variables:
    streamflow_max_spring  (id, lmom) float64 1kB 318.3 182.7 ... 0.2504 0.191
    streamflow_max_annual  (id, lmom) float64 1kB 396.2 104.8 ... 0.2089 0.2656
    volume_sum_spring      (id, lmom) float64 1kB 337.9 56.86 ... 0.1032 -0.0149
    volume_sum_annual      (id, lmom) float64 1kB 1.132e+03 170.1 ... 0.3158

In [80]:
ds_groups = xr.concat(
    [
        ds_4fa.sel(id=groupes[i]).assign_coords(group_id=i).expand_dims("group_id")
        for i in range(len(groupes))
    ],
    dim="group_id",
)
ds_moments_groups = xr.concat(
    [
        ds_moment.sel(id=groupes[i]).assign_coords(group_id=i).expand_dims("group_id")
        for i in range(len(groupes))
    ],
    dim="group_id",
)

In [81]:
ds_moments_groups.sel(group_id=0).dropna(dim="id", how="all")

<xarray.Dataset> Size: 5kB
Dimensions:                (id: 16, lmom: 6)
Coordinates: (12/16)
  * id                     (id) object 128B '022507' '022601' ... '024015'
  * lmom                   (lmom) <U4 96B 'l1' 'l2' 'l3' 'tau' 'tau3' 'tau4'
    group_id               int32 4B 0
    drainage_area          (id) float32 64B 515.0 213.0 796.0 ... 227.0 48.1
    end_date               (id) datetime64[ns] 128B 2024-05-30 ... 2024-05-28
    latitude               (id) float32 64B 47.61 47.53 47.38 ... 46.05 46.18
    ...                     ...
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 128B 1978-09-09 ... 2006-07-24
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
Data variables:
    streamflow_max_spring  (id, lmom) float64 768B 96.97 17.99 ... 0.2504 0.191
    streamflow_max_annual  (id, lmom) float64 768B 98.97 18.44 ... 0.2089 0.2656
    volume_sum_spring      (id, lmom) float64 768B 84.53 17.42 ... -0.0149
    volume_sum_annual      (id, lmom) float64 768B 306.9 34.27 ... 0.3158

In [82]:
ds_groups.sel(group_id=0).dropna(dim="id", how="all")

<xarray.Dataset> Size: 23kB
Dimensions:                (id: 16, time: 56)
Coordinates: (12/16)
  * id                     (id) object 128B '022507' '022601' ... '024015'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
    group_id               int32 4B 0
    drainage_area          (id) float32 64B 515.0 213.0 796.0 ... 227.0 48.1
    end_date               (id) datetime64[ns] 128B 2024-05-30 ... 2024-05-28
    latitude               (id) float32 64B 47.61 47.53 47.38 ... 46.05 46.18
    ...                     ...
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 128B 1978-09-09 ... 2006-07-24
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
Data variables:
    streamflow_max_spring  (id, time) float32 4kB nan nan nan ... 11.29 nan nan
    streamflow_max_annual  (id, time) float32 4kB nan nan nan ... 15.12 nan nan
    volume_sum_spring      (id, time) float64 7kB nan nan nan ... 1.907 nan nan
    volume_sum_annual      (id, time) float64 7kB nan nan nan ... 37.17 nan nan
Attributes:
    cat:variable:          ('streamflow_max_spring',)
    cat:xrfreq:            YS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [163]:
kap = KappaGen()
ds_H_Z = calc_h_z(ds_groups, ds_moments_groups, kap)
ds_H_Z

<xarray.Dataset> Size: 732B
Dimensions:                (group_id: 3, crit: 2)
Coordinates:
  * group_id               (group_id) int32 12B 0 1 2
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
    lmom                   <U4 16B 'l1'
  * crit                   (crit) <U1 8B 'H' 'Z'
Data variables:
    streamflow_max_spring  (crit, group_id) float64 48B -0.551 nan ... nan nan
    streamflow_max_annual  (crit, group_id) float64 48B 0.2605 ... -0.3275
    volume_sum_spring      (crit, group_id) float64 48B 7.813 2.278 ... 0.2188
    volume_sum_annual      (crit, group_id) float64 48B 19.67 nan ... -0.1175

In [104]:
ds_H = combine_H_Z(ds_H_Z)
ds_H

<xarray.Dataset> Size: 732B
Dimensions:                (group_id: 3, crit: 2)
Coordinates:
  * group_id               (group_id) int32 12B 0 1 2
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
    lmom                   <U4 16B 'l1'
  * crit                   (crit) <U1 8B 'H' 'Z'
Data variables:
    streamflow_max_spring  (crit, group_id) float64 48B -0.551 nan ... nan nan
    streamflow_max_annual  (crit, group_id) float64 48B 0.2605 ... -0.3275
    volume_sum_spring      (crit, group_id) float64 48B 7.813 2.278 ... 0.2188
    volume_sum_annual      (crit, group_id) float64 48B 19.67 nan ... -0.1175

In [105]:
mask = mask_h_z(ds_H)

In [106]:
ds_groups_H1 = ds_groups.where(mask).load()
ds_moments_groups_H1 = ds_moments_groups.where(mask).load()

In [108]:
centiles = [x / 100.0 for x in range(101)]
return_periods = [
    1.010101,
    1.052632,
    1.111111,
    1.25,
    1.5,
    2,
    3,
    5,
    10,
    20,
    50,
    100,
    200,
    500,
    1000,
    2000,
    5000,
    10000,
]

In [121]:
Q_T = calculate_rp_from_AFR(ds_groups_H1, ds_moments_groups_H1, return_periods)
Q_T = remove_small_regions(Q_T)

In [122]:
Q_T

<xarray.Dataset> Size: 35kB
Dimensions:                (group_id: 2, rp: 18, id: 27)
Coordinates: (12/17)
  * group_id               (group_id) int32 8B 0 1
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
    ...                     ...
    latitude               (group_id, id) float32 216B nan nan nan ... nan nan
    longitude              (group_id, id) float32 216B nan nan nan ... nan nan
    name                   (group_id, id) object 432B nan nan nan ... nan nan
    province               (group_id, id) object 432B nan nan nan ... nan nan
    regulated              (group_id, id) object 432B nan nan nan ... nan nan
    start_date             (group_id, id) datetime64[ns] 432B NaT NaT ... NaT
Data variables:
    streamflow_max_spring  (group_id, rp, id) float64 8kB nan nan ... nan nan
    streamflow_max_annual  (group_id, rp, id) float64 8kB nan nan ... nan nan
    volume_sum_spring      (group_id, rp, id) float64 8kB nan nan ... nan nan
    volume_sum_annual      (group_id, rp, id) float64 8kB nan nan ... nan nan

In [123]:
Q = Q_T.sel(id="023401").dropna(dim="group_id", how="all")

In [124]:
Q

<xarray.Dataset> Size: 1kB
Dimensions:                (group_id: 1, rp: 18)
Coordinates: (12/17)
  * group_id               (group_id) int32 4B 0
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
    ...                     ...
    latitude               (group_id) float32 4B 46.66
    longitude              (group_id) float32 4B -71.29
    name                   (group_id) object 8B 'Beaurivage'
    province               (group_id) object 8B 'QC'
    regulated              (group_id) object 8B 'Natural'
    start_date             (group_id) datetime64[ns] 8B 1970-01-01
Data variables:
    streamflow_max_spring  (group_id, rp) float64 144B 81.78 105.1 ... 592.0
    streamflow_max_annual  (group_id, rp) float64 144B 91.46 114.4 ... 576.8
    volume_sum_spring      (group_id, rp) float64 144B nan nan nan ... nan nan
    volume_sum_annual      (group_id, rp) float64 144B nan nan nan ... nan nan

In [150]:
Q_T = calculate_rp_from_AFR(ds_groups_H1, ds_moments_groups_H1, return_periods)
Q_T = remove_small_regions(Q_T)
Q = Q_T.sel(id="023401").dropna(dim="group_id", how="all")
Q

<xarray.Dataset> Size: 1kB
Dimensions:                (group_id: 1, rp: 18)
Coordinates: (12/17)
  * group_id               (group_id) int32 4B 0
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
    ...                     ...
    latitude               (group_id) float32 4B 46.66
    longitude              (group_id) float32 4B -71.29
    name                   (group_id) object 8B 'Beaurivage'
    province               (group_id) object 8B 'QC'
    regulated              (group_id) object 8B 'Natural'
    start_date             (group_id) datetime64[ns] 8B 1970-01-01
Data variables:
    streamflow_max_spring  (group_id, rp) float64 144B 81.78 105.1 ... 592.0
    streamflow_max_annual  (group_id, rp) float64 144B 91.46 114.4 ... 576.8
    volume_sum_spring      (group_id, rp) float64 144B nan nan nan ... nan nan
    volume_sum_annual      (group_id, rp) float64 144B nan nan nan ... nan nan